In [ ]:
pip install pennylane

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive') #lo concecto con mi drive

import cv2
import os
import numpy as np
import torch
import torchvision
from torchvision import models  #Parte TL
from torch.nn.functional import relu


from torch import nn

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
directory = './data'
if not os.path.exists(directory):
    os.mkdir(directory)

# Descomprimir el zip con el dataset
with zipfile.ZipFile("./drive/MyDrive/Colab Notebooks/QuantumKernelsPlayDogsVSCats/datasetCatsVSDogs.zip","r") as z:
    z.extractall("./data")

data_dir = './data/datasetCatsVSDogs'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D

In [ ]:
base_model = VGG16(weights='imagenet')
model = tf.keras.Model(inputs=base_model.input,      
outputs=base_model.get_layer('block5_pool').output)
#model.summary()
modelAux = Sequential()
modelAux.add(model)
modelAux.summary()

In [ ]:
#layer19 = model.layers[18]

modelAux.add(Conv2D(64, (1,1), activation='relu'))
modelAux.add(Conv2D(32, (1,1), activation='relu'))
modelAux.add(MaxPooling2D(pool_size=(2, 2)))
modelAux.add(Conv2D(16, (1,1), activation='relu'))
modelAux.add(Conv2D(8, (1,1), activation='relu'))
modelAux.add(Conv2D(4, (1,1), activation='relu'))
modelAux.add(MaxPooling2D(pool_size=(2, 2)))
modelAux.add(Flatten())

In [ ]:
modelAux.summary()

In [ ]:
import PIL

In [ ]:
train_dir = "./data/datasetCatsVSDogs/train"

categorias = ['cats', 'dogs']

dataset = []

def get_features(img_path):
    img = load_img(img_path, target_size=(224, 224))
    #img = cv2.imread(imgpath,1) # el 0 indica que se cargue la imagen en escala de grises, un 1 indica en color
    #img = cv2.resize( img, (224,224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0) #la dimension de x era 3 y pasa a ser 4
    x = preprocess_input(x)
    flatten = modelAux.predict(x) #la dimension de flatten es 2
    print(flatten[0])
    return list(flatten[0])

features, labels = [], []

for categoria in categorias:
  path = os.path.join(train_dir, categoria) #ruta de la carpeta cats o la carpeta dogs
  label = categorias.index(categoria)
  i=0
  for img in os.listdir(path):
    if i <100:
      imgpath = os.path.join(path, img) #ruta de la imagen
      try:
          img = PIL.Image.open(imgpath)
      except PIL.UnidentifiedImageError:
              print(imgpath) #imprime el nombre de la imagen si se produce fallo al cargar alguna
      if (imgpath != "./data/datasetCatsVSDogs/train/cats/_DS_Store") & (imgpath != "./data/datasetCatsVSDogs/train/dogs/_DS_Store"): #este archivo esta en la carpeta pero no es una de las imagenes
        features.append(get_features(imgpath))
        labels.append(label)
    i += 1

In [ ]:
print(len(features))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( features, labels, test_size=0.15, shuffle= True)

In [ ]:
#n_qubits = len(X_train[0])
#n_qubits = len(X_train[0])
n_qubits = 4
print(n_qubits)

In [ ]:
dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [ ]:
kernel(X_train[0], X_train[0])

In [ ]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [ ]:
modelQuantumKernel = SVC(kernel=kernel_matrix, probability = True).fit(X_train, y_train)

In [ ]:
predictions = modelQuantumKernel.predict(X_test) #predice sobre el test set

In [ ]:
print(accuracy_score(predictions, y_test)) #calcula la precision

In [ ]:
pip install dill

In [ ]:
import dill

In [ ]:
torch.save(modelQuantumKernel, 'VGG16_QSVM_definitivo.pth', pickle_module=dill) #necesario dill para poder cargarlo en vs code

In [ ]:
loaded_model = SVC(kernel=kernel_matrix, probability = True)
loaded_model = torch.load('VGG16_QSVM_definitivo.pth',map_location=device, pickle_module=dill)

In [ ]:
imagePath = './data/datasetCatsVSDogs/test/cats/cat.4001.jpg'

imagenToPredict=[]

imagenToPredict.append(get_features(imagePath)) #le debo aplicar a la imagen el mismo preprocesado que a las que se usaron para entrenar

In [ ]:
prediccion = loaded_model.predict_proba(imagenToPredict)

#print('La predicción es:', predictedClass[0], ', ', categorias[predictedClass[0]])

In [ ]:
print(prediccion)

In [ ]:
count = 0
for i in prediccion[0]:
  if count == 0:
    catProbability = i
  else:
    dogProbability = i
  count += 1


if catProbability > dogProbability:
  classPredicted = "Cat"
else:
  classPredicted = "Dog"
  
print('La predicción es:', classPredicted)

